In [20]:
! unzip '/content/All Midi Files.zip' #unzipping file

Archive:  /content/All Midi Files.zip
   creating: All Midi Files/albeniz/
  inflating: All Midi Files/albeniz/alb_esp1.mid  
  inflating: All Midi Files/albeniz/alb_esp2.mid  
  inflating: All Midi Files/albeniz/alb_esp3.mid  
  inflating: All Midi Files/albeniz/alb_esp4.mid  
  inflating: All Midi Files/albeniz/alb_esp5.mid  
  inflating: All Midi Files/albeniz/alb_esp6.mid  
  inflating: All Midi Files/albeniz/alb_se1.mid  
  inflating: All Midi Files/albeniz/alb_se2.mid  
  inflating: All Midi Files/albeniz/alb_se3.mid  
  inflating: All Midi Files/albeniz/alb_se4.mid  
  inflating: All Midi Files/albeniz/alb_se5.mid  
  inflating: All Midi Files/albeniz/alb_se6.mid  
  inflating: All Midi Files/albeniz/alb_se7.mid  
  inflating: All Midi Files/albeniz/alb_se8.mid  
   creating: All Midi Files/bach/
  inflating: All Midi Files/bach/bach_846.mid  
  inflating: All Midi Files/bach/bach_847.mid  
  inflating: All Midi Files/bach/bach_850.mid  
   creating: All Midi Files/balakir/
  in

# Importing

In [21]:
from music21 import *
import glob
from tqdm import tqdm
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import os

In [23]:
from tensorflow.keras.layers import LSTM,Dense,Input,Dropout
from tensorflow.keras.models import Sequential,Model,load_model 

In [24]:
import tensorflow as tf


# Data Preprocssing

In [25]:
def file_reader(file_path):
    notes=[]
    notes_to_parse=None
    #parse the midi file
    midi=converter.parse(file_path)
    #seperate all instruments from the file
    instrmt=instrument.partitionByInstrument(midi)
    for part in instrmt.parts:
        #fetch data only of Piano instrument
        if 'Piano' in str(part):
            notes_to_parse=part.recurse()
            for element in notes_to_parse:
                if type(element)==note.Note:
                    notes.append(str(element.pitch))
                elif type(element)==chord.Chord:
                    notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes
#retrieve paths recursively from inside the directories/files
file_path=['haydn','grieg'] 
all_files=glob.glob('/content/All Midi Files/'+file_path[0]+'/*.mid',recursive=True)
notes_array = np.array([file_reader(i) for i in all_files])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [26]:
#unique notes
notess = sum(notes_array,[]) 
unique_notes = list(set(notess))
print("Unique Notes:",len(unique_notes))

#notes with their frequency
freq=dict(map(lambda x: (x,notess.count(x)),unique_notes))

#get the threshold frequency
print("\nFrequency notes")
for i in range(30,100,20):
  print(i,":",len(list(filter(lambda x:x[1]>=i,freq.items()))))

#filter notes greater than threshold i.e. 50
freq_notes=dict(filter(lambda x:x[1]>=50,freq.items()))

#create new notes using the frequent notes
new_notes=[[i for i in j if i in freq_notes] for j in notes_array]

#dictionary having key as note index and value as note
ind2note=dict(enumerate(freq_notes))

#dictionary having key as note and value as note index
note2ind=dict(map(reversed,ind2note.items()))

Unique Notes: 155

Frequency notes
30 : 76
50 : 64
70 : 56
90 : 48


In [27]:
timesteps=50

#store values of input and output
x=[] ; y=[]

for i in new_notes:
  for j in range(0,len(i)-timesteps):
    #input will be the current index + timestep
    #output will be the next index after timestep
    inp=i[j:j+timesteps] ; out=i[j+timesteps]

    #append the index value of respective notes 
    x.append(list(map(lambda x:note2ind[x],inp)))
    y.append(note2ind[out])

x_new=np.array(x) 
y_new=np.array(y)

In [28]:
#reshape input and output for the model
x_new = np.reshape(x_new,(len(x_new),timesteps,1))
y_new = np.reshape(y_new,(-1,1))

#split the input and value into training and testing sets
x_train,x_test,y_train,y_test = train_test_split(x_new,y_new,test_size=0.2,random_state=42)

In [29]:
x_train.shape

(20313, 50, 1)

# Model Buiding

In [30]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [31]:
import tensorflow as tf

In [32]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

In [33]:
#create the model
model = Sequential()
#create two stacked LSTM layer with the latent dimension of 256
model.add(LSTM(300,return_sequences=True,input_shape=(x_new.shape[1],x_new.shape[2])))
model.add(Dropout(0.22))
model.add(LSTM(300))
model.add(Dropout(0.22))
model.add(Dense(300,activation='relu'))

#fully connected layer for the output with softmax activation
model.add(Dense(len(note2ind),activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 300)           362400    
                                                                 
 dropout (Dropout)           (None, 50, 300)           0         
                                                                 
 lstm_1 (LSTM)               (None, 300)               721200    
                                                                 
 dropout_1 (Dropout)         (None, 300)               0         
                                                                 
 dense (Dense)               (None, 300)               90300     
                                                                 
 dense_1 (Dense)             (None, 64)                19264     
                                                                 
Total params: 1,193,164
Trainable params: 1,193,164
Non-

In [34]:
#compile the model using Adam optimizer
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

#train the model on training sets and validate on testing sets
with tf.device('/gpu:0'):
    model.fit(
        x_train,y_train,
        batch_size=100,epochs=60, 
        validation_data=(x_test,y_test))

Epoch 1/60
204/204 [==============================] - 12s 24ms/step - loss: 3.7554 - accuracy: 0.0633 - val_loss: 3.6963 - val_accuracy: 0.0772
Epoch 2/60
204/204 [==============================] - 4s 20ms/step - loss: 3.6316 - accuracy: 0.0854 - val_loss: 3.5540 - val_accuracy: 0.0982
Epoch 3/60
204/204 [==============================] - 4s 20ms/step - loss: 3.4753 - accuracy: 0.1120 - val_loss: 3.4096 - val_accuracy: 0.1134
Epoch 4/60
204/204 [==============================] - 4s 20ms/step - loss: 3.3474 - accuracy: 0.1336 - val_loss: 3.2945 - val_accuracy: 0.1433
Epoch 5/60
204/204 [==============================] - 4s 20ms/step - loss: 3.1837 - accuracy: 0.1596 - val_loss: 3.2138 - val_accuracy: 0.1603
Epoch 6/60
204/204 [==============================] - 4s 20ms/step - loss: 3.0212 - accuracy: 0.1957 - val_loss: 3.0043 - val_accuracy: 0.1993
Epoch 7/60
204/204 [==============================] - 4s 20ms/step - loss: 2.8137 - accuracy: 0.2358 - val_loss: 2.8668 - val_accuracy: 0.234

In [35]:
model.save("Model")

INFO:tensorflow:Assets written to: Model/assets


INFO:tensorflow:Assets written to: Model/assets


In [36]:
model=load_model("Model")
#generate random index
index = np.random.randint(0,len(x_test)-1)
#get the data of generated index from x_test
music_pattern = x_test[index]

out_pred=[] #it will store predicted notes

#iterate till 200 note is generated
for i in range(200):

  #reshape the music pattern 
  music_pattern = music_pattern.reshape(1,len(music_pattern),1)
  
  #get the maximum probability value from the predicted output
  pred_index = np.argmax(model.predict(music_pattern))
  #get the note using predicted index and
  #append to the output prediction list
  out_pred.append(ind2note[pred_index])
  music_pattern = np.append(music_pattern,pred_index)
  
  #update the music pattern with one timestep ahead
  music_pattern = music_pattern[1:]

In [37]:
output_notes = []
for offset,pattern in enumerate(out_pred):
  #if pattern is a chord instance
  if ('.' in pattern) or pattern.isdigit():
    #split notes from the chord
    notes_in_chord = pattern.split('.')
    notes = []
    for current_note in notes_in_chord:
        i_curr_note=int(current_note)
        #cast the current note to Note object and
        #append the current note 
        new_note = note.Note(i_curr_note)
        new_note.storedInstrument = instrument.Piano()
        notes.append(new_note)
    
    #cast the current note to Chord object
    #offset will be 1 step ahead from the previous note
    #as it will prevent notes to stack up 
    new_chord = chord.Chord(notes)
    new_chord.offset = offset
    output_notes.append(new_chord)
  
  else:
    #cast the pattern to Note object apply the offset and 
    #append the note
    new_note = note.Note(pattern)
    new_note.offset = offset
    new_note.storedInstrument = instrument.Piano()
    output_notes.append(new_note)

#save the midi file 
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='AI_music.mid')

'AI_music.mid'